In [1]:
#!/usr/bin/env python3
"""
Multi-area image-level CV of ViT–Brain alignment + per-area 'significant images'.

- Loads ViT logits -> softmax -> CLR -> PCA(90%)
- For each brain area:
    * builds trial-averaged neuron x image matrix -> PCA(90%)
    * K-fold image-level CV of cross-domain shared spectrum (train vs test)
    * picks best fold (highest test shared var[0])
    * visualizes held-out images ranked by shared PC1 (top/bottom 10)
    * saves all figures to OUT_DIR/<AREA>_*.png
    * saves numpy results for later analysis

Author: Maria + Pläku 🐾
"""

import os, pickle, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA
from scipy.special import softmax
from skbio.stats.composition import clr
from numpy.random import default_rng
from PIL import Image

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'
IMAGES_DIR  = '/home/maria/MITNeuralComputation/vit_embeddings/images'   # scene_000.png ... scene_117.png
OUT_DIR     = '/home/maria/MITNeuralComputation/vit_multi_area_cv'

N_IMAGES, N_TRIALS = 118, 50
VAR_CUTOFF  = 0.90
N_SPLITS    = 5
RANDOM_SEED = 42
AREA_WHITELIST = None   # e.g. ['VISp','VISam','VISpm'] or None for all found

rng = default_rng(RANDOM_SEED)
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# -----------------------------
# HELPERS
# -----------------------------
def crossdomain_svd(Zv, Zb):
    Zv = Zv - Zv.mean(axis=0, keepdims=True)
    Zb = Zb - Zb.mean(axis=0, keepdims=True)
    C  = (Zv.T @ Zb) / Zv.shape[0]
    U, S, Vt = np.linalg.svd(C, full_matrices=False)
    return U, S, Vt, (S**2) / np.sum(S**2)

def grid_images(indices, title, savepath):
    plt.figure(figsize=(12,2.2))
    for i, idx in enumerate(indices):
        img = Image.open(os.path.join(IMAGES_DIR, f"scene_{idx:03d}.png"))
        plt.subplot(1, len(indices), i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.savefig(savepath, dpi=160)
    plt.close()

def scatter_plot(x, y, title, savepath):
    r = np.corrcoef(x, y)[0,1]
    plt.figure(figsize=(4,4))
    plt.scatter(x, y, alpha=0.75)
    plt.xlabel("ViT projection on shared PC1 (test)")
    plt.ylabel("Brain projection on shared PC1 (test)")
    plt.title(f"{title} (r={r:.2f})")
    plt.tight_layout()
    plt.savefig(savepath, dpi=160)
    plt.close()
    return r

# -----------------------------
# 1) ViT preprocess (softmax -> CLR -> PCA@90%)
# -----------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']  # (118, D_vit)

Xv = softmax(np.asarray(vit_logits), axis=1)
Xv = clr(Xv + 1e-12)

vit_pca_full = PCA(random_state=RANDOM_SEED).fit(Xv)
vit_ncomp = np.searchsorted(np.cumsum(vit_pca_full.explained_variance_ratio_), VAR_CUTOFF) + 1
vit_pca = PCA(n_components=vit_ncomp, random_state=RANDOM_SEED)
Zv      = vit_pca.fit_transform(Xv)   # (images × vit_ncomp)
print(f"✅ ViT PCs covering 90% variance: {vit_ncomp}")

# -----------------------------
# 2) Load neural + area labels once
# -----------------------------
dat_all = np.load(NEURAL_PATH, mmap_mode='r')  # (neurons × (images*trials*time))
areas   = np.load(AREAS_PATH, allow_pickle=True)
all_areas = np.unique(areas)
if AREA_WHITELIST:
    all_areas = [a for a in all_areas if a in AREA_WHITELIST]
print(f"Found areas: {all_areas}")

# -----------------------------
# 3) Image-level CV per area
# -----------------------------
indices = np.arange(Zv.shape[0])
rng.shuffle(indices)
folds = np.array_split(indices, N_SPLITS)

summary = {}

for AREA_NAME in all_areas:
    print(f"\n=== {AREA_NAME} ===")

    # --- build neurons×images matrix (trial-avg)
    mask = (areas == AREA_NAME)
    dat  = dat_all[mask]
    n_neurons, n_total = dat.shape
    n_time = n_total // (N_IMAGES * N_TRIALS)
    dat    = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
    X_mean = dat.mean(axis=(2,3))                                  # (neurons × images)

    # --- PCA@90% on brain
    brain_pca_full = PCA(random_state=RANDOM_SEED).fit(X_mean.T)
    brain_ncomp = np.searchsorted(np.cumsum(brain_pca_full.explained_variance_ratio_), VAR_CUTOFF) + 1
    brain_pca = PCA(n_components=brain_ncomp, random_state=RANDOM_SEED)
    Zb        = brain_pca.fit_transform(X_mean.T)                  # (images × brain_ncomp)
    print(f"{AREA_NAME}: PCs@90% = {brain_ncomp}")

    # --- cross-validated spectra
    spectra_train, spectra_test, fold_meta = [], [], []
    for f, test_idx in enumerate(folds):
        train_idx = np.setdiff1d(indices, test_idx)

        Zv_train, Zb_train = Zv[train_idx], Zb[train_idx]
        Zv_test,  Zb_test  = Zv[test_idx],  Zb[test_idx]

        # Train shared subspace on TRAIN images
        U_tr, S_tr, Vt_tr, shared_train = crossdomain_svd(Zv_train, Zb_train)

        # Project TEST images into learned subspace
        Zv_proj = Zv_test @ U_tr
        Zb_proj = Zb_test @ Vt_tr.T

        # Test shared spectrum
        _, S_te, _, shared_test = crossdomain_svd(Zv_proj, Zb_proj)

        spectra_train.append(shared_train)
        spectra_test.append(shared_test)
        fold_meta.append((U_tr, Vt_tr, train_idx, test_idx))
        print(f"  Fold {f+1}: test shared var[0]={shared_test[0]:.3f}")

    spectra_train = np.array(spectra_train)
    spectra_test  = np.array(spectra_test)
    mean_train = spectra_train.mean(axis=0)
    mean_test  = spectra_test.mean(axis=0)
    std_test   = spectra_test.std(axis=0)

    # --- save spectrum plot
    x = np.arange(1, len(mean_test)+1)
    plt.figure(figsize=(6,4))
    plt.bar(x, mean_test, yerr=std_test, color='tomato', alpha=0.75, capsize=3, label='test')
    plt.plot(x, mean_train, color='gray', linestyle='--', label='train')
    plt.xlabel("Shared component index")
    plt.ylabel("Shared variance fraction")
    plt.title(f"Image-level CV ViT–{AREA_NAME} ({N_SPLITS}-fold)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{AREA_NAME}_spectrum.png"), dpi=160)
    plt.close()

    # --- pick best fold (max test shared var[0]) and visualize test images
    best_fold = int(np.argmax(spectra_test[:, 0]))
    U_tr, Vt_tr, train_idx, test_idx = fold_meta[best_fold]
    Zv_test, Zb_test = Zv[test_idx], Zb[test_idx]

    vit_scores   = Zv_test @ U_tr[:, 0]
    brain_scores = Zb_test @ Vt_tr.T[:, 0]

    # Normalize for plotting
    vit_scores   = (vit_scores - vit_scores.mean()) / (vit_scores.std() + 1e-8)
    brain_scores = (brain_scores - brain_scores.mean()) / (brain_scores.std() + 1e-8)

    # rank held-out images (by ViT projection)
    order = np.argsort(vit_scores)
    top10 = order[-10:]
    bot10 = order[:10]

    # save grids + scatter
    grid_images([test_idx[i] for i in top10],
                f"Top +10 test images along shared PC1 ({AREA_NAME})",
                os.path.join(OUT_DIR, f"{AREA_NAME}_test_top10.png"))
    grid_images([test_idx[i] for i in bot10],
                f"Bottom −10 test images along shared PC1 ({AREA_NAME})",
                os.path.join(OUT_DIR, f"{AREA_NAME}_test_bot10.png"))

    r = scatter_plot(vit_scores, brain_scores,
                     f"ViT–{AREA_NAME} shared PC1 (held-out)",
                     os.path.join(OUT_DIR, f"{AREA_NAME}_test_scatter.png"))

    # --- save numpy bundle for the area
    np.savez(os.path.join(OUT_DIR, f"{AREA_NAME}_results.npz"),
             spectra_train=spectra_train,
             spectra_test=spectra_test,
             mean_train=mean_train, mean_test=mean_test, std_test=std_test,
             best_fold=best_fold, test_idx=test_idx,
             vit_scores=vit_scores, brain_scores=brain_scores, r_test=r,
             vit_pcs=vit_pca.components_, brain_pcs=brain_pca.components_)

    summary[AREA_NAME] = dict(
        mean_test_first=float(mean_test[0]),
        std_test_first=float(std_test[0]),
        r_test=float(r),
        n_neurons=int(n_neurons),
        brain_pcs=int(brain_ncomp),
        vit_pcs=int(vit_ncomp),
        best_fold=int(best_fold)
    )

# -----------------------------
# 4) Print compact summary table
# -----------------------------
print("\n===== Summary (per area) =====")
for a, s in summary.items():
    print(f"{a:6s} | test var[0]={s['mean_test_first']:.3f}±{s['std_test_first']:.3f} | "
          f"r_test={s['r_test']:.2f} | neurons={s['n_neurons']} | PCs: brain={s['brain_pcs']}, ViT={s['vit_pcs']} | best_fold={s['best_fold']}")
print(f"\nFigures/results saved in: {OUT_DIR}")


🔹 Loading ViT embeddings...
✅ ViT PCs covering 90% variance: 44
Found areas: ['VISal' 'VISam' 'VISl' 'VISp' 'VISpm' 'VISrl']

=== VISal ===
VISal: PCs@90% = 78
  Fold 1: test shared var[0]=0.407
  Fold 2: test shared var[0]=0.422
  Fold 3: test shared var[0]=0.592
  Fold 4: test shared var[0]=0.386
  Fold 5: test shared var[0]=0.368

=== VISam ===
VISam: PCs@90% = 81
  Fold 1: test shared var[0]=0.325
  Fold 2: test shared var[0]=0.411
  Fold 3: test shared var[0]=0.574
  Fold 4: test shared var[0]=0.482
  Fold 5: test shared var[0]=0.318

=== VISl ===
VISl: PCs@90% = 83
  Fold 1: test shared var[0]=0.311
  Fold 2: test shared var[0]=0.485
  Fold 3: test shared var[0]=0.472
  Fold 4: test shared var[0]=0.356
  Fold 5: test shared var[0]=0.306

=== VISp ===
VISp: PCs@90% = 87
  Fold 1: test shared var[0]=0.412
  Fold 2: test shared var[0]=0.541
  Fold 3: test shared var[0]=0.443
  Fold 4: test shared var[0]=0.401
  Fold 5: test shared var[0]=0.343

=== VISpm ===
VISpm: PCs@90% = 83
  Fo